In [1]:
import pandas as pd
import re
import math
import requests

In [3]:
df = pd.read_csv('phishing_site_urls.csv')
df.head()

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad


In [4]:
tldSet = set()
word_frequency = {}
sensitive_words = []

In [5]:
def remove_special_characters(input_string):
  # Pattern to match any non-alphanumeric and non-space character
  pattern = r'[^a-zA-Z0-9\s]'  
  return re.sub(pattern, '', input_string)

In [ ]:
def calculate_entropy(word):
  if len(word) == 0:
    return 0

  entropy = 0
  word_length = len(word)
  char_count = {}

  for char in word:
    if char in char_count:
      char_count[char] += 1
    else:
      char_count[char] = 1

  for char in char_count:
    probability = char_count[char] / word_length
    entropy += -probability * math.log2(probability)

  return entropy

In [7]:
# Iterate all rows using DataFrame.iterrows()
for index, row in df.iterrows():
  # if(index == 250):
  #   break

  URL = row["URL"]
  domain = URL.split("/")[0]
  tld = domain.split(".")[-1]
  tld_length = len(tld)
  url_length = len(URL)

  tldSet.add(tld)

  if row["Label"] == "bad":
    temp = URL.split("/")
    subdomain = "/".join(temp[1:])

    words_list = re.split("[/,.,?,-,+,=,&,~,!,@,#,$,%,_,;]", subdomain)
    for word in words_list:
      word = word.strip().lower()
      word = remove_special_characters(word)
      digs = len(re.findall('[0-9]', word))
      alps = len(re.findall('[A-z]', word))
      if(len(word) < 2 or len(word) > 15 or digs > alps):
        continue
      if word in word_frequency:
        word_frequency[word] += 1
      else:
        word_frequency[word] = 1

In [ ]:
sorted_word_frequency = sorted(word_frequency.items(), key=lambda x:x[1], reverse=True)

for it in sorted_word_frequency:
  if(it[1] > 5):
    sensitive_words.append(it)

#### *** Export Sensitive Words to a file. Only the words sensitive_words[0] *** ###
#### *** Loop through good label rows to export their TLD to a file *** ###

In [ ]:
for it in sensitive_words:
  print(it[0], " : ", it[1], "\n")

In [ ]:
with open('output.txt', 'w') as file:
  for item in sensitive_words:
    file.write(str(item) + '\n')

In [ ]:
# https://www.whoisxmlapi.com/whoisserver/WhoisService?apiKey=at_BGLU4RpM0fcKHpqsqHAYRWFl4bIt2&domainName=facebook.com

# API Key: at_BGLU4RpM0fcKHpqsqHAYRWFl4bIt2
# Rapid API Key: d2592fe48dmsh190428470dadf9cp1bf762jsnf40816e6115e (Shauryansingh23102001@gm)

url = "https://whoisapi-whois-v2-v1.p.rapidapi.com/whoisserver/WhoisService"

querystring = {"domainName":"codechef.com",
              "apiKey":"at_BGLU4RpM0fcKHpqsqHAYRWFl4bIt2",
              "outputFormat":"JSON",
              "da":"0",
              "ipwhois":"1",
              "thinWhois":"0",
              "_parse":"0",
              "preferfresh":"1",
              "checkproxydata":"0",
              "ip":"1"
              }

headers = {
	"X-RapidAPI-Key": "d2592fe48dmsh190428470dadf9cp1bf762jsnf40816e6115e",
	"X-RapidAPI-Host": "whoisapi-whois-v2-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

### Important Attributes From WHOIS API
---
createdDate: Date Format String

updatedDate: Date Format String

expiresDate: Date Format String

registrant.countryCode : String

registrarName: String

domainNameExt: String

estimatedDomainAge : Int

ips : Array[STRING]

registrarIANAID

Nameservers: 

Cheack if nameservers are from reputable hosting providers

In [ ]:
response_text = response.text

In [ ]:
with open("output2.json", "w") as file:
  # Write the response content to the file
  file.write(response_text)